In [ ]:
import pandas as pd

In [ ]:
# profile of school account
df_profile = pd.read_csv("KYRC24_ACCT_Profile.csv")

In [ ]:
# remove unnecessary columns and put focus on high schools only
df_profile = df_profile[(df_profile['Level'] == 'High School') & (df_profile['School Name']!= 'All Schools')]
df_profile = df_profile.drop(['School Year', 'County Number', 'District Number', 'School Number', 'School Code', 'State School Id', 'NCES Id', 'Co-Op', 'Co-Op Code','Federal Classification', 'Reason for Federal Classification', 'Feeder School'], axis=1)

In [ ]:
# remove repetitive data
def remove_repeats(df):
    dropped = []
    for col in df_profile.columns:
        if 'CHANGE DIFFERENCE' in col:
            dropped.append(col)
    return df_profile.drop(dropped, axis = 1)
    
df_profile = remove_repeats(df_profile)